<a href="https://colab.research.google.com/github/efrsw/hsebio-hw3/blob/main/hsebio_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Part 1